In [1]:
import pandas as pd
import os
import numpy as np
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
import re
from sklearn.impute import KNNImputer
import pickle

/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/pandas/core/computation/expressions.py:21: UserWarning: Pandas requires version '2.8.0' or newer of 'numexpr' (version '2.7.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED


# Handling Errant Values 

In [3]:
raw_file = "/home/ec2-user/SageMaker/Team-5/Submission/Processing data/TN/TN_allinone0112.csv"
df = pd.read_csv(raw_file)

/tmp/ipykernel_28500/790404473.py:2: DtypeWarning: Columns (1,2,3,4,5,6,8,11,14,16,17,18,19,20,23,25,26,27,28,29,30,31,33,35,37,42,44,48,49,51,52,54,55,57,59,61,63,65,66,69,70,73,74,77,80,81,84,85,86,87,90,92,93,94,95,97,102,103,104,105,106,107,109,112,115,116,117,119,120,128,141,142,143,144,145,146,147,149,151,154,158,160,162,163,167,168,172,173,177,178,182,183,187,188,189,190,191,192,194,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,256,257,258,259,260,261,262,263,264,265,266,267,268,272,275,279,280,281,282,283,284,285,286,287,288,289,290,291,292,293,294,295,296,297,298,299,300,303,304,305,306,307,308,309,311,314,317,319,321,323,324,326,328,329,331,334,336,339,341,342,344,345,347,348,349,350,352,356,358,359,361,363,365,366,368,370,372,373,378,380,382,384,385,386,387,388,389,390,391,392,393,395,397,398,399,400,401,402

In [4]:
df

,MaskID,Visit_Dt,Visit,_22Jul2009VisitType,_15Aug2011VisitType,StorageOfResidualsDuringTrialN,_15Aug2011ConsentForResidualSa,_22Jul2009ConsentSignedDay,_22Jul2009ConsentSignedMonth,_22Jul2009ConsentSignedYear,...,CREA,CHL,BIC,ALT,ALP,ALB,UREA,AST,CPEP,INST
0,200082,03/30/2010,PRN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,200082,06/13/2012,PRN,NaN,Rescreen/Confirmatory Screening Visit,NaN,Yes,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,200185,07/06/2016,PRN,NaN,Initial Screening Visit,NaN,Yes,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,200273,05/16/2012,PRN,NaN,Initial Screening Visit,Not applicable,Yes,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,200273,06/14/2018,PRN,NaN,Rescreen/Confirmatory Screening Visit,NaN,Yes,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195593,996198,05/25/2016,2 weeks,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
195594,996198,10/13/2016,20 weeks,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
195595,996198,10/27/2016,22 weeks,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
195596,996198,11/08/2016,24 weeks,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 195598 entries, 0 to 195597
Columns: 1049 entries, MaskID to INST
dtypes: float64(427), int64(1), object(621)
memory usage: 1.5+ GB


In [6]:
## Converting heights
df['Heightin_to_cm'] = df['Heightin'] * 2.53999863
df['Height_cm'] = df['Heightin_to_cm'] + df['Heightcm']
df.insert(df.columns.get_loc('Height_cm') + 1, 'Height_cm_units', 'cm')
df.drop(['Heightin_to_cm', 'Heightcm', 'Heightin'], axis=1, inplace=True)

df['Weightlbs_to_kg'] = df['Weightlbs'] * 0.45359237
df['Weight_kg'] = df['Weightlbs_to_kg'] + df['Weightkg']
df.insert(df.columns.get_loc('Weight_kg') + 1, 'Weight_kg_units', 'kg')
df.drop(['Weightlbs_to_kg', 'Weightkg', 'Weightlbs'], axis=1, inplace=True)

In [7]:
## Converting glucose units and organizing columns
for column in df.columns:
    if 'mmol' in column.lower() or 'mgdl' in column.lower():
        new_column_name = column.replace('mmol', '_unit').replace('mgdl', '').replace('Mmoll', '_unit').replace('MgDL', '').replace('mmoll', '_unit')
        df.rename(columns={column: new_column_name}, inplace=True)
        
columns_to_process = ['AvgRecordedFastingGlucoses', 'AvgAllRecordedGlucoses', 'LowestGlucose', 'HighestGlucose']
df[columns_to_process] = df[columns_to_process].apply(lambda x: x.where(x >= 45, x * 18.018))

columns_to_convert = ['AvgRecordedFastingGlucoses_unit', 'AvgAllRecordedGlucoses_unit', 'LowestGlucose_unitl', 'HighestGlucose_unit']
df[columns_to_convert] = df[columns_to_convert].replace('mmol/L', 'mg/dl')

column_to_move1 = 'AvgRecordedFastingGlucoses_unit'
target_position1 = 'AvgRecordedFastingGlucoses'
current_positions = df.columns.tolist()
current_positions.remove(column_to_move1)
target_index = current_positions.index(target_position1)
new_positions = current_positions[:target_index + 1] + [column_to_move1] + current_positions[target_index + 1:]
df = df[new_positions]

column_to_move2 = 'AvgAllRecordedGlucoses_unit'
target_position2 = 'AvgAllRecordedGlucoses'
current_positions = df.columns.tolist()
current_positions.remove(column_to_move2)
target_index = current_positions.index(target_position2)
new_positions = current_positions[:target_index + 1] + [column_to_move2] + current_positions[target_index + 1:]
df = df[new_positions]

df.replace('mg/dl', 'mg/dL', inplace=True, regex=True)

In [8]:
## Applying middle 99.5% filter, excluding two columns, to identify outstanding values
numeric_columns = df.select_dtypes(include=np.number).columns

columns_to_skip = ['Rbcresults', 'HemoglobinResults','MaskID']
for column in numeric_columns:
    if column not in columns_to_skip:
        lower_bound = df[column].quantile(0.0025)
        upper_bound = df[column].quantile(0.9975)
        df[column] = np.where((df[column] < lower_bound) | (df[column] > upper_bound), np.nan, df[column])

RBC_filtered_rows = df[df['RBCResultsNormal'] == 'Yes']
Hemoglobin_filtered_rows = df[df['HemoglobinResultsNormal'] == 'Yes']

RBC_condition = 'RBCResultsNormal'
RBC_filter = 'Rbcresults'
filtered_rows = df[df[RBC_condition] == 'Yes'].copy()
lower_bound = filtered_rows[RBC_filter].quantile(0.0025)
upper_bound = filtered_rows[RBC_filter].quantile(0.9975)
filtered_rows.loc[
    (filtered_rows[RBC_filter] < lower_bound) | (filtered_rows[RBC_filter] > upper_bound),
    RBC_filter] = np.nan

Hemoglobin_condition = 'HemoglobinResultsNormal'
Hemoglobin_filter = 'HemoglobinResults'
filtered_rows = df[df[Hemoglobin_condition] == 'Yes'].copy()
lower_bound = filtered_rows[Hemoglobin_filter].quantile(0.0025)
upper_bound = filtered_rows[Hemoglobin_filter].quantile(0.9975)
filtered_rows.loc[
    (filtered_rows[Hemoglobin_filter] < lower_bound) | (filtered_rows[Hemoglobin_filter] > upper_bound),
    Hemoglobin_filter] = np.nan

In [9]:
df.head()

,MaskID,Visit_Dt,Visit,_22Jul2009VisitType,_15Aug2011VisitType,StorageOfResidualsDuringTrialN,_15Aug2011ConsentForResidualSa,_22Jul2009ConsentSignedDay,_22Jul2009ConsentSignedMonth,_22Jul2009ConsentSignedYear,...,ALP,ALB,UREA,AST,CPEP,INST,Height_cm,Height_cm_units,Weight_kg,Weight_kg_units
0,200082,03/30/2010,PRN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,cm,NaN,kg
1,200082,06/13/2012,PRN,NaN,Rescreen/Confirmatory Screening Visit,NaN,Yes,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,NaN,cm,NaN,kg
2,200185,07/06/2016,PRN,NaN,Initial Screening Visit,NaN,Yes,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,NaN,cm,NaN,kg
3,200273,05/16/2012,PRN,NaN,Initial Screening Visit,Not applicable,Yes,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,NaN,cm,NaN,kg
4,200273,06/14/2018,PRN,NaN,Rescreen/Confirmatory Screening Visit,NaN,Yes,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,cm,NaN,kg


In [10]:
print(df.info())
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 195598 entries, 0 to 195597
Columns: 1049 entries, MaskID to Weight_kg_units
dtypes: float64(425), int64(1), object(623)
memory usage: 1.5+ GB
None


,MaskID,Visit_Dt,Visit,_22Jul2009VisitType,_15Aug2011VisitType,StorageOfResidualsDuringTrialN,_15Aug2011ConsentForResidualSa,_22Jul2009ConsentSignedDay,_22Jul2009ConsentSignedMonth,_22Jul2009ConsentSignedYear,...,ALP,ALB,UREA,AST,CPEP,INST,Height_cm,Height_cm_units,Weight_kg,Weight_kg_units
0,200082,03/30/2010,PRN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,cm,NaN,kg
1,200082,06/13/2012,PRN,NaN,Rescreen/Confirmatory Screening Visit,NaN,Yes,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,NaN,cm,NaN,kg
2,200185,07/06/2016,PRN,NaN,Initial Screening Visit,NaN,Yes,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,NaN,cm,NaN,kg
3,200273,05/16/2012,PRN,NaN,Initial Screening Visit,Not applicable,Yes,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,NaN,cm,NaN,kg
4,200273,06/14/2018,PRN,NaN,Rescreen/Confirmatory Screening Visit,NaN,Yes,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,cm,NaN,kg


In [11]:
# Replace both "." and " ." with NaN
df = df.replace({'.': np.nan, ' .': np.nan, '  .':np.nan})

# Date column harmonization

# 1. Columns of Date

#### Steps of date column harmonization:
1. Filter out date column sets based on their headers
2. Create one date column to replace one set of date columns.
3. assign values in the format of date_time to the new created column.

In [12]:
merged_df=df
#This is a tool to find columns based on names of their headers and check their values. 
contain_list_0 = ['SerumKetonesDay'] 
# Example strings to exclude
not_contain_list_0 = []     

# Converting each string in the lists to lower case
contain_list = [string.lower() for string in contain_list_0]
not_contain_list = [string.lower() for string in not_contain_list_0]

# Filtering columns with case-insensitive comparison
filtered_columns = [col for col in merged_df.columns if 
                    any(contain in col.lower() for contain in contain_list) and 
                    all(not_contain not in col.lower() for not_contain in not_contain_list)]

print("Found ", len(filtered_columns), "columns: ", filtered_columns)

filtered_df = merged_df[filtered_columns].dropna(how='all')

# Output the filtered DataFrame
print(filtered_df.info())

# Uncomment if you want to print top 20 values for each column
for col in filtered_columns:
    print(f"Top 20 values for column {col}:")
    print(merged_df[col].value_counts().nlargest(20))
    print()

Found  0 columns:  []
<class 'pandas.core.frame.DataFrame'>
Index: 0 entries
Empty DataFrame
None


## 1.1 Day, Month, Year

In [13]:
# Filter all columns with "Day", "Month", "Year" as date sets
# in lower case for case-insensitive comparison
# Example strings to include
contain_list_1 = ['Day', "Month", 'Year'] 
# Example strings to exclude
not_contain_list_1 = ['LastYear', 'Last6Months', '3Days', 'Birth', 'MostRecentHbA1c', 'PastYear', 
                      'ChangeInDrugStatusEffective', 'ParticipantDateDispensed', 'MechanisticSampleCollection',
                     'Polydipsia', 'Polyuria', '10days', 'ADHPsoriasisDateofDiagnosis', 'CeliacSprueGlutenDiagnosis',
                     'Datethebloodsamplewasdrawn', 'MHInflammatoryBowelDisease', 'ADHAlopeciaDateofDiagnosis', 
                      'ADHOtherDateofDiagnosis', 'ADHVitiligoDateofDiagnosis', 'VitiligoDiagnosis', 'NextYear',
                      'Polyphagia', 'Fatigue', 'monthly', 'Ketoacidosis', 'UnexplainedWeightLoss', 'HbA1crecordDate']     

# Converting each string in the lists to lower case
contain_list_1_low = [string.lower() for string in contain_list_1]
not_contain_list_1_low = [string.lower() for string in not_contain_list_1]

# Filtering columns with case-insensitive comparison
filtered_date_columns_1 = [col for col in df.columns if 
                    any(contain in col.lower() for contain in contain_list_1_low) and 
                    all(not_contain not in col.lower() for not_contain in not_contain_list_1_low)]

print("Found ", len(filtered_date_columns_1), "columns: ", filtered_date_columns_1)

filtered_df = df[filtered_date_columns_1]

# Output the filtered DataFrame
print(filtered_df.info())

# Uncomment if you want to print top 20 values for each column
#for col in filtered_columns:
#    print(f"Top 20 values for column {col}:")
#    print(df[col].value_counts().nlargest(20))
#    print()

Found  42 columns:  ['Spring2019ConsentDateMonth', 'Spring2019ConsentDateDay', 'Spring2019ConsentDateYear', 'DateOfBloodDrawDay', 'DateOfBloodDrawMonth', 'DateOfBloodDrawYear', 'DateReportedDay', 'DateReportedMonth', 'DateReportedYear', 'ScreeningDay', 'ScreeningMonth', 'ScreeningYear', 'ConsentSignedDay', 'ConsentSignedMonth', 'ConsentSignedYear', 'DateConsentSignedDay', 'DateConsentSignedMonth', 'DateConsentSignedYear', 'SamplesCollectionDay', 'SamplesCollectionMonth', 'SamplesCollectionYear', 'LastHbA1cDay', 'LastHbA1cMonth', 'LastHbA1cYear', 'WrittenInformedConsentDay', 'WrittenInformedConsentMonth', 'WrittenInformedConsentYear', 'Type1DiabetesDiagnosisMonth', 'Type1DiabetesDiagnosisDay', 'Type1DiabetesDiagnosisYear', 'FluVaccinationDateMonth', 'FluVaccinationDateDay', 'FluVaccinationDateYear', 'DateTestsRunMonth', 'DateTestsRunDay', 'DateTestsRunYear', 'StudyDrugReturnedDay', 'StudyDrugReturnedMonth', 'StudyDrugReturnedYear', 'StudyDrugDispensedDay', 'StudyDrugDispensedMonth', 'St

#### Check month and year columns based on day columns

In [14]:
#check columns for the day
day_column_list_1 = [col for col in filtered_date_columns_1 if "Day" in col]
print("No. of the day columes to be merged: ", len(day_column_list_1))

#create columns for the month and the year based on columns of the day
month_column_list_1 = [name.replace("Day", "Month") for name in day_column_list_1]
year_column_list_1 = [name.replace("Day", "Year") for name in day_column_list_1]

#check whether there are any columns in the dataframe not following the naming rules
unmatched_month_columns = [name for name in month_column_list_1 if name not in df.columns]
print("No. of month columes in wrong naming format:", len(unmatched_month_columns))
print("Month columes in wrong naming format:", unmatched_month_columns)

unmatched_year_columns = [name for name in year_column_list_1 if name not in df.columns]
print("No. of year columes in wrong naming format: ", len(unmatched_year_columns))
print("Year columes in wrong naming format: ", unmatched_year_columns)

No. of the day columes to be merged:  14
No. of month columes in wrong naming format: 0
Month columes in wrong naming format: []
No. of year columes in wrong naming format:  0
Year columes in wrong naming format:  []


#### Check day and year columns based on month columns

In [15]:
#check columns for the day
month_column_list_1_1 = [col for col in filtered_date_columns_1 if "Month" in col]
print("No. of the day columes to be merged: ", len(month_column_list_1_1))

#create columns for the month and the year based on columns of the day
day_column_list_1_1 = [name.replace("Month", "Day") for name in month_column_list_1_1]
year_column_list_1_1 = [name.replace("Month", "Year") for name in month_column_list_1_1]

#check whether there are any columns in the dataframe not following the naming rules
unmatched_day_columns = [name for name in day_column_list_1_1 if name not in df.columns]
print("No. of day columes in wrong naming format:", len(unmatched_day_columns))
print("Day columes in wrong naming format:", unmatched_day_columns)

unmatched_year_columns = [name for name in year_column_list_1_1 if name not in df.columns]
print("No. of year columes in wrong naming format: ", len(unmatched_year_columns))
print("Year columes in wrong naming format: ", unmatched_year_columns)

No. of the day columes to be merged:  14
No. of day columes in wrong naming format: 0
Day columes in wrong naming format: []
No. of year columes in wrong naming format:  0
Year columes in wrong naming format:  []


In [16]:
import numpy as np
# Convert 'NA' strings to np.nan
def convert_na(value):
    if pd.isna(value) or value == 'NA':  # using pandas function to check for NA
        return np.nan   # converting to numpy NaN
    else:
        return value

# Function to convert month from string or number to numeric month, handling missing values
def month_to_num(month):
    if pd.isna(month):
        return np.nan
    try:
        # Handle string representation
        if isinstance(month, str):
            datetime_object = pd.to_datetime(month, format='%b')
            return datetime_object.month
        # Handle numeric representation
        elif isinstance(month, (int, float)):
            return int(month)
    except ValueError:
        return np.nan
    
# Create a date column and assign values by combine date columns with similar header
def create_date_column(df, year_col, original_cols):
    parts = year_col.split('Year')
    base_col_name = parts[0]
    suffix = parts[1] if len(parts) > 1 else ''

    day_col = base_col_name + 'Day' + suffix
    month_col = base_col_name + 'Month' + suffix

    if day_col in original_cols and month_col in original_cols:
        date_col = base_col_name + 'Date' + suffix
        df[date_col] = pd.NaT  # Initialize column with NaT (Not-a-Time) for datetime objects
        problematic_rows = []

        for index, row in df.iterrows():
            try:
                if pd.isna(row[year_col]) or pd.isna(row[month_col]) or pd.isna(row[day_col]):
                    continue  # NaNs are already handled by NaT initialization
                else:
                    year = int(row[year_col])
                    month = int(month_to_num(row[month_col]))
                    day = int(row[day_col])
                    df.at[index, date_col] = pd.to_datetime(f"{year}-{month:02d}-{day:02d}", errors='coerce').date()
            except (ValueError, OverflowError) as e:
                problematic_rows.append((index, row[year_col], row[month_col], row[day_col]))
                df.at[index, date_col] = pd.NaT

        if problematic_rows:
            print("Problematic rows:")
            for row in problematic_rows:
                print(row)
    else:
        print(f"Matching columns for '{year_col}' not found.")

In [17]:
# Convert 'NA' strings to np.nan
for col in filtered_date_columns_1:
    df.loc[:, col] = df[col].apply(convert_na)

# Iterating over columns and creating date columns
for col in year_column_list_1:
    create_date_column(df, col, filtered_date_columns_1)

print(df.info())
df.head()

/tmp/ipykernel_33758/1799681261.py:46: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '2019-07-22' has dtype incompatible with datetime64[ns], please explicitly cast to a compatible dtype first.
  df.at[index, date_col] = pd.to_datetime(f"{year}-{month:02d}-{day:02d}", errors='coerce').date()
/tmp/ipykernel_33758/1799681261.py:46: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '2018-11-08' has dtype incompatible with datetime64[ns], please explicitly cast to a compatible dtype first.
  df.at[index, date_col] = pd.to_datetime(f"{year}-{month:02d}-{day:02d}", errors='coerce').date()
/tmp/ipykernel_33758/1799681261.py:46: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '2018-11-08' has dtype incompatible with datetime64[ns], please explicitly cast to a compatible dtype first.
  df.at

KeyboardInterrupt: 

In [ ]:
df.info()

In [ ]:
columns_to_drop = [col for col in filtered_date_columns_1]

print(len(columns_to_drop))
df.drop(columns=columns_to_drop, inplace=True)
df.info()

## 1.2 Month, Year

In [ ]:
column_pairs = [
    ('BirthMonth', 'BirthYear'),
    ('DateOfBirthMonth', 'DateOfBirthYear'),
    ('HbA1crecordDateMonth', 'HbA1crecordDateYear')
]

In [ ]:
# Modified convert_to_datetime function to handle missing columns
def convert_to_datetime(row, month_col, year_col):
    try:
        if pd.notna(row[month_col]) and pd.notna(row[year_col]):
            month = row[month_col].capitalize()  # Capitalize to match standard abbreviations
            year = int(row[year_col])
            return pd.to_datetime(f"{month} {year}", format='%b %Y').strftime('%b %Y')
        else:
            return np.nan
    except (ValueError, TypeError):
        return np.nan
    
def safe_convert_to_datetime(df, month_col, year_col):
    if month_col in df.columns and year_col in df.columns:
        return df.apply(lambda row: convert_to_datetime(row, month_col, year_col), axis=1)
    else:
        return pd.Series([np.nan] * len(df))

# Create a new DataFrame for the new columns
new_columns_df = pd.DataFrame()

In [ ]:
# Iterate over each pair and create a new column
for month_col, year_col in column_pairs:
    new_col_name = month_col.replace('Month', '') + 'Date'
    new_columns_df[new_col_name] = safe_convert_to_datetime(df, month_col, year_col)

# Check for missing columns and print them
for month_col, year_col in column_pairs:
    if month_col not in df.columns or year_col not in df.columns:
        print(f"Column pair missing: {month_col}, {year_col}")

# Concatenate the new columns with the original DataFrame
df = pd.concat([df, new_columns_df], axis=1)

# Check the result
print(df.info())
df.head()

In [ ]:
#df = df.drop(['MostRecentHbA1cDate'], axis=1)

In [ ]:
df.info()

In [ ]:
columns_to_drop = [col for pair in column_pairs for col in pair]

# Drop the columns from the DataFrame
df.drop(columns=columns_to_drop, inplace=True)
df.info()

## 1.3 DD, MM, YYYY

In [ ]:
# Filter all columns with "DD", "MM", or "YYYY"
contain_list_1 = ["DD", "MM", "YYYY"] 
# Example strings to exclude
not_contain_list_1 = ['Autoimmuned', 'MMTT', 'TTT', 'TimeOfCollection', 'TimeResultsProcessedByLab']     

# Converting each string in the lists to lower case
#contain_list_1_low = [string.lower() for string in contain_list_1]
#not_contain_list_1_low = [string.lower() for string in not_contain_list_1]

# Filtering columns with case-insensitive comparison
filtered_date_columns_1 = [col for col in df.columns if 
                    any(contain in col for contain in contain_list_1) and 
                    all(not_contain not in col for not_contain in not_contain_list_1)]

print("Found ", len(filtered_date_columns_1), "columns: ", filtered_date_columns_1)

filtered_df = df[filtered_date_columns_1]

# Output the filtered DataFrame
print(filtered_df.info())

In [ ]:
#check columns for the day
day_column_list_1 = [col for col in filtered_date_columns_1 if "DD" in col]
print("No. of the day columes to be merged: ", len(day_column_list_1))

#create columns for the month and the year based on columns of the day
month_column_list_1 = [name.replace("DD", "MM") for name in day_column_list_1]
year_column_list_1 = [name.replace("DD", "YYYY") for name in day_column_list_1]

#check whether there are any columns in the dataframe not following the naming rules
unmatched_month_columns = [name for name in month_column_list_1 if name not in df.columns]
print("No. of month columes in wrong naming format:", len(unmatched_month_columns))
print("Month columes in wrong naming format:", unmatched_month_columns)

unmatched_year_columns = [name for name in year_column_list_1 if name not in df.columns]
print("No. of year columes in wrong naming format: ", len(unmatched_year_columns))
print("Year columes in wrong naming format: ", unmatched_year_columns)

In [ ]:
#check columns for the day
month_column_list_1_1 = [col for col in filtered_date_columns_1 if "MM" in col]
print("No. of the day columes to be merged: ", len(month_column_list_1_1))

#create columns for the month and the year based on columns of the day
day_column_list_1_1 = [name.replace("MM", "DD") for name in month_column_list_1_1]
year_column_list_1_1 = [name.replace("MM", "YYYY") for name in month_column_list_1_1]

#check whether there are any columns in the dataframe not following the naming rules
unmatched_day_columns = [name for name in day_column_list_1_1 if name not in df.columns]
print("No. of day columes in wrong naming format:", len(unmatched_day_columns))
print("Day columes in wrong naming format:", unmatched_day_columns)

unmatched_year_columns = [name for name in year_column_list_1_1 if name not in df.columns]
print("No. of year columes in wrong naming format: ", len(unmatched_year_columns))
print("Year columes in wrong naming format: ", unmatched_year_columns)

In [ ]:
import pandas as pd
import numpy as np

# Function to convert month from string to number, handling missing values
def month_to_num_1(month_str):
    if pd.isna(month_str):
        return np.nan
    try:
        datetime_object = pd.to_datetime(month_str, format='%b')
        return datetime_object.month
    except ValueError:
        return np.nan
    
def create_date_column_2(df, year_col, original_cols):
    parts = year_col.split('YYYY')
    base_col_name = parts[0]
    suffix = parts[1] if len(parts) > 1 else ''

    day_col = base_col_name + 'DD' + suffix
    month_col = base_col_name + 'MM' + suffix

    if day_col in original_cols and month_col in original_cols:
        date_col = base_col_name + 'Date' + suffix
        problematic_rows = []
        for index, row in df.iterrows():
            try:
                if pd.isna(row[year_col]) or pd.isna(row[month_col]) or pd.isna(row[day_col]):
                    df.at[index, date_col] = np.nan
                else:
                    year = int(row[year_col])
                    month = int(month_to_num_1(row[month_col]))
                    day = int(row[day_col])
                    datetime_obj = pd.to_datetime(f"{year}-{month:02d}-{day:02d}")
                    df.at[index, date_col] = datetime_obj.strftime('%Y-%m-%d')  # Format to only include the date
            except (ValueError, OverflowError) as e:
                problematic_rows.append((index, row[year_col], row[month_col], row[day_col]))
                df.at[index, date_col] = np.nan
        if problematic_rows:
            print("Problematic rows:")
            for row in problematic_rows:
                print(row)
    else:
        print(f"Matching columns for '{year_col}' not found.")

In [ ]:
# Convert 'NA' strings to np.nan
for col in filtered_date_columns_1:
    df[col] = df[col].apply(convert_na)

original_cols = df.columns.tolist()

# Iterating over columns and creating date columns
for col in filtered_date_columns_1:
    if 'YYYY' in col:
        create_date_column_2(df, col, original_cols)

print(df.info())
df.head()

In [ ]:
columns_to_drop = [col for col in filtered_date_columns_1]

len(columns_to_drop)

# Drop the columns from the DataFrame
df.drop(columns=columns_to_drop, inplace=True)
df.info()

## 1.4 day, mon, yea

In [ ]:
contain_list_0 = ['mon'] 
# Example strings to exclude
not_contain_list_0 = ['month', 'mont', 'Monitor', 'Mono', 'PastSixMon', 'Whendidthepneumoniaoccur', 'ReasonsNotInterested', 'Monetary']     

# Converting each string in the lists to lower case
contain_list = [string.lower() for string in contain_list_0]
not_contain_list = [string.lower() for string in not_contain_list_0]

# Filtering columns with case-insensitive comparison
filtered_columns = [col for col in merged_df.columns if 
                    any(contain in col.lower() for contain in contain_list) and 
                    all(not_contain not in col.lower() for not_contain in not_contain_list)]

print("Found ", len(filtered_columns), "columns: ", filtered_columns)

filtered_df = merged_df[filtered_columns].dropna(how='all')

# Output the filtered DataFrame
print(filtered_df.info())

# Uncomment if you want to print top 20 values for each column
for col in filtered_columns:
    print(f"Top 20 values for column {col}:")
    print(merged_df[col].value_counts().nlargest(20))
    print()

In [ ]:
col_list = ['ParticipantDateDispensedDay1_1', 'ParticipantDateDispensedMon1_1', 'ParticipantDateDispensedYea1_1',
'ParticipantDateDispensedDay2_1', 'ParticipantDateDispensedMon2_1', 'ParticipantDateDispensedYea2_1',
'ParticipantDateDispensedDay3_1', 'ParticipantDateDispensedMon3_1', 'ParticipantDateDispensedYea3_1',
'MechanisticSampleCollectionDay', 'MechanisticSampleCollectionMon', 'MechanisticSampleCollectionYea']

In [ ]:
def create_date_column_3(df, year_col, original_cols):
    parts = year_col.split('Yea')
    base_col_name = parts[0]
    suffix = parts[1] if len(parts) > 1 else ''

    day_col = base_col_name + 'Day' + suffix
    month_col = base_col_name + 'Mon' + suffix

    if day_col in original_cols and month_col in original_cols:
        date_col = base_col_name + 'Date' + suffix
        problematic_rows = []
        for index, row in df.iterrows():
            try:
                if pd.isna(row[year_col]) or pd.isna(row[month_col]) or pd.isna(row[day_col]):
                    df.at[index, date_col] = np.nan
                else:
                    year = int(row[year_col])
                    month = int(month_to_num_1(row[month_col]))
                    day = int(row[day_col])
                    datetime_obj = pd.to_datetime(f"{year}-{month:02d}-{day:02d}")
                    df.at[index, date_col] = datetime_obj.strftime('%Y-%m-%d')  # Format to only include the date
            except (ValueError, OverflowError) as e:
                problematic_rows.append((index, row[year_col], row[month_col], row[day_col]))
                df.at[index, date_col] = np.nan
        if problematic_rows:
            print("Problematic rows:")
            for row in problematic_rows:
                print(row)
    else:
        print(f"Matching columns for '{year_col}' not found.")

In [ ]:
# Convert 'NA' strings to np.nan
for col in col_list:
    df[col] = df[col].apply(convert_na)

original_cols = df.columns.tolist()

# Iterating over columns and creating date columns
for col in col_list:
    if 'Yea' in col:
        create_date_column_3(df, col, original_cols)

print(df.info())
df.head()

In [ ]:
len(col_list)

# Drop the columns from the DataFrame
df.drop(columns=col_list, inplace=True)
df.info()

## 1.5 D, M, Y

In [ ]:

def create_date_from_components(row):
    # Extracting year, month, and day from the row
    year = row['DateSignedInformedConsentFormY']
    month = row['DateSignedInformedConsentFormM']
    day = row['DateSignedInformedConsentFormD']

    # Check for NaN values and convert floats to integers
    if pd.isna(year) or pd.isna(month) or pd.isna(day):
        return pd.NaT
    try:
        year = int(year)
        day = int(day)
        # Constructing the date string
        date_str = f"{year}-{month}-{day:02d}"
        # Converting to datetime
        return pd.to_datetime(date_str, format='%Y-%b-%d').date()
    except (ValueError, TypeError):
        return pd.NaT

# Apply the function to each row to create the new date column
df['DateSignedInformedConsentFormDate'] = df.apply(create_date_from_components, axis=1)

In [ ]:
df.info()

In [ ]:
# Drop the columns from the DataFrame
df.drop(columns=['DateSignedInformedConsentFormM', 'DateSignedInformedConsentFormD', 'DateSignedInformedConsentFormY'], inplace=True)
df.info()

## 1.6 Columns without indication of day, month, year

In [ ]:
date_dict_1 ={ 'DateSignedNEWInformedConsentFoDate': ['DateSignedNEWInformedConsentFo', 'DateSignedNEWInformedConsentFo2', 'DateSignedNEWInformedConsentFo3'],
              'InterferonGammaReleaseAssayIGRDate': ['InterferonGammaReleaseAssayIGR', 'InterferonGammaReleaseAssayIGR3', 'InterferonGammaReleaseAssayIGR2']
             }

In [ ]:
# Function to convert month from short name to number
def month_to_num_2(short_month):
    return {
        'Jan': 1, 'Feb': 2, 'Mar': 3, 'Apr': 4, 'May': 5, 'Jun': 6,
        'Jul': 7, 'Aug': 8, 'Sep': 9, 'Oct': 10, 'Nov': 11, 'Dec': 12
    }[short_month]

# Iterating over the dictionary to create new columns in filtered_df
for date_col, cols in date_dict_1.items():
    day_col, month_col, year_col = cols

    # Create a mask for rows where none of the day, month, or year values are missing
    mask = df[[day_col, month_col, year_col]].notna().all(axis=1)

    # Apply the mask to filter out rows with missing values
    valid_rows = df[mask]

    # Iterate over each row to handle individual errors
    for index, row in valid_rows.iterrows():
        try:
            # Combine the date components for the current row
            combined_date = str(int(row[day_col])) + '/' + \
                            str(month_to_num_2(row[month_col])) + '/' + \
                            str(int(row[year_col]))
            
            # Convert to datetime
            df.at[index, date_col] = pd.to_datetime(combined_date, format='%d/%m/%Y')
        except ValueError as e:
            # Print the problematic date and its index
            print(f"Error at index {index}: {combined_date}")
            print(f"Error message: {e}")

# Display the new DataFrame
print(df.info())
df.head()

In [ ]:
columns_to_drop = [column for sublist in date_dict_1.values() for column in sublist]
print(len(columns_to_drop))

# Drop the columns from the DataFrame
df.drop(columns=columns_to_drop, inplace=True)
df.info()

## 1.7 time

In [ ]:
time_dict = {'OGTTTestStartTime': ['OGTTTestStartTimeHrs', 'OGTTTestStartTimeMins'],
             'MMTTTestStartTime': ['MMTTTestStartTimeHrs', 'MMTTTestStartTimeMins'],
             'TimeOfCollection': ['TimeOfCollectionHH', 'TimeOfCollectionMM'],
             'TimeResultsProcessedByLab': ['TimeResultsProcessedByLabHH', 'TimeResultsProcessedByLabMM']
            }

In [ ]:
def create_time_columns(df, time_dict):
    for new_col, time_cols in time_dict.items():
        hour_col, minute_col = time_cols

        if hour_col in df.columns and minute_col in df.columns:
            # Creating a time column with handling missing values and converting to integers
            df[new_col] = df.apply(lambda row: 
                                   f"{int(row[hour_col]):02d}:{int(row[minute_col]):02d}"
                                   if pd.notna(row[hour_col]) and pd.notna(row[minute_col]) and 
                                      row[hour_col] != '.' and row[minute_col] != '.'
                                   else np.nan, axis=1)
            # Converting string to datetime.time object
            df[new_col] = pd.to_datetime(df[new_col], format='%H:%M', errors='coerce').dt.time
        else:
            print(f"Columns for '{new_col}' not found. Hour column: {hour_col}, Minute column: {minute_col}")

    return df

df = create_time_columns(df, time_dict)
print(df.info())
df.head()

In [ ]:
columns_to_drop = [column for sublist in time_dict.values() for column in sublist]
print(len(columns_to_drop))

# Drop the columns from the DataFrame
df.drop(columns=columns_to_drop, inplace=True)
df.info()

In [ ]:
df

In [78]:
df.to_csv('df_column_harmonization.csv', index = False)

In [149]:
df=pd.read_csv('df_column_harmonization.csv')

/tmp/ipykernel_33758/821926730.py:1: DtypeWarning: Columns (5,8,9,11,12,14,15,17,18,20,22,23,25,27,28,31,32,35,36,39,40,43,44,47,48,49,50,51,52,53,54,55,56,57,58,59,96,97,101,102,103,125,126,130,131,135,136,140,143,145,147,148,149,150,151,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,171,172,173,174,177,178,179,180,181,182,184,189,191,193,195,196,197,198,200,204,205,206,207,208,209,211,212,213,214,215,244,245,251,252,254,255,256,257,258,260,262,263,264,265,269,271,273,275,276,279,280,282,283,284,286,289,292,293,295,296,297,298,299,300,308,309,310,311,312,313,314,315,316,317,318,319,320,321,322,323,324,325,326,327,328,329,330,331,332,333,334,335,336,337,338,339,340,341,343,344,345,346,440,441,442,443,444,445,446,447,448,449,450,451,452,453,457,464,468,469,470,473,474,475,476,477,478,479,480,486,487,488,494,495,496,497,498,499,500,501,502,503,504,505,508,512,515,516,518,520,522,523,524,525,526,527,532,533,534,535,536,622,623,628,629,630,631,632,633,634,636,637,638,639,640,6

In [150]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 259064 entries, 0 to 259063
Columns: 652 entries, MaskID to TimeResultsProcessedByLab
dtypes: float64(311), int64(1), object(340)
memory usage: 1.3+ GB


# Missing values

In [147]:
df.dropna(axis=1, how='all', inplace=True)
df.dropna(axis=0, how='all', inplace=True)

In [180]:
#identify categorical columns and numeric columns first 

categorical_columns = df.select_dtypes(include=['object', 'category']).columns
numerical_columns = df.select_dtypes(include=['int', 'float']).columns
date_columns = df.select_dtypes(include=['datetime64']).columns

print("Categorical columns:", categorical_columns)
print('number of categorical columns:',len(categorical_columns))
print("Numerical columns:", numerical_columns) 
print('number of numerical columns:',len(numerical_columns))
print('number of date columns:',len(date_columns))

Categorical columns: Index(['Visit_Dt', 'Specimencollectionkitshipmentd', 'Date_of_Registration',
       'RememberDateOfMostRecentHbA1c', 'DateOfRegistration',
       'DiabeticKetoacidosisTimeOfDiag', 'MethylprednisoneGivenDoseTimeM',
       'Spring2019ConsentDateDate', 'DateOfBloodDrawDate', 'DateReportedDate',
       'ScreeningDate', 'ConsentSignedDate', 'DateConsentSignedDate',
       'SamplesCollectionDate', 'LastHbA1cDate', 'WrittenInformedConsentDate',
       'Type1DiabetesDiagnosisDate', 'FluVaccinationDateDate',
       'DateTestsRunDate', 'StudyDrugReturnedDate', 'StudyDrugDispensedDate',
       'BirthDate', 'DateOfBirthDate', 'HbA1crecordDateDate',
       'DateRecordedDate1', 'DateRecordedDate2', 'DateRecordedDate3',
       'ParticipantDateDispensedDate1_1', 'ParticipantDateDispensedDate2_1',
       'ParticipantDateDispensedDate3_1', 'MechanisticSampleCollectionDate',
       'DateSignedInformedConsentFormDate',
       'DateSignedNEWInformedConsentFoDate',
       'InterferonGam

In [22]:
#1st handle wrong datatype
#for any categorical variable, if it contains more than 20 unique values, then we can use common sense to judge 
# if it is better to convert it to 'int' datatype.
for i in categorical_columns:
    if len(df[i].unique())>20:
        print(i)
        print(df[i].unique())

Visit
['Screening' '1 year' 'Inital Visit' '12 months' '18 months' '24 months'
 '30 months' '6 months' '1 year, 6 months' '2 years' '2 years, 6 months'
 '3 years' '4 years' '3 years, 6 months' '4 years, 6 months' '5 years'
 '5 years, 6 months' '6 years' '6 years, 6 months' '7 years'
 '7 years, 6 months' '8 years' '8 years, 6 months' '11 years'
 '11 years, 6 months' '12 years, 6 months' '13 years' '1 month' '2 months'
 '3 months' '7 months' '8 months' '9 months' 'Day 1' '9 years, 6 months'
 '10 years' '9 years' '36 months' '42 months' '48 months' '54 months'
 '10 years, 6 months' '60 months' '66 months' '72 months' '78 months'
 '12 years' '13 years, 6 months' '14 years' '15 years' nan
 '14 years, 6 months' '15 years, 6 months']
Visit_Dt
['06/29/2007' '06/29/2015' '08/11/2015' ... '10/24/2010' '08/28/2011'
 '04/26/2009']
InitialScreeningLocationOtherS
[nan 'JDRF ONE NATION' 'Screening event' 'Bay Area Diabetes Summit'
 'Offsite Screening Event' 'Offsite' 'Family Diabetes Event'
 'T1D Con

Date_of_Registration
[nan '03/31/2016' '03/17/2016' '04/14/2016' '05/02/2016' '12/06/2016'
 '04/11/2016' '12/15/2015' '08/04/2016' '08/12/2016' '04/20/2016'
 '05/23/2016' '06/29/2016' '10/14/2016' '05/06/2016' '07/15/2016'
 '12/23/2016' '07/14/2016' '08/01/2016' '01/11/2017' '07/06/2016'
 '08/17/2016' '11/11/2016' '07/21/2016' '04/13/2016' '07/22/2016'
 '06/24/2016' '12/21/2016' '05/11/2016' '10/24/2016' '06/14/2016'
 '08/03/2016' '10/18/2016' '01/05/2016' '07/11/2016' '05/31/2016'
 '12/19/2016' '11/04/2016' '08/19/2016' '06/21/2016' '12/12/2016'
 '05/13/2016' '09/28/2016' '06/06/2016' '08/09/2016' '05/12/2016'
 '10/05/2016' '06/17/2016' '09/13/2016' '05/10/2016' '09/19/2016'
 '04/25/2016' '12/20/2016' '03/25/2016' '05/17/2016' '04/27/2016'
 '05/04/2016' '04/21/2016' '12/16/2016' '07/05/2016' '12/22/2016'
 '09/06/2016' '09/12/2016' '04/07/2016' '04/06/2016' '09/14/2016'
 '01/31/2017' '01/03/2017' '12/28/2015' '12/13/2016' '05/25/2016'
 '06/09/2016' '06/05/2016' '10/17/2016' '06/02/2016

[nan '2017-05-04' '2017-11-02' '2018-11-02' '2016-06-01' '2019-08-12'
 '2015-03-24' '2016-01-22' '2016-07-28' '2017-01-27' '2017-06-15'
 '2018-03-16' '2019-01-22' '2014-12-27' '2015-07-02' '2017-06-20'
 '2017-08-15' '2018-07-20' '2016-08-09' '2015-03-30' '2016-07-18'
 '2017-08-24' '2018-04-12' '2014-02-28' '2014-09-19' '2013-06-07'
 '2015-09-22' '2017-12-19' '2018-07-17' '2019-02-05' '2017-05-30'
 '2015-12-08' '2016-09-30' '2015-04-27' '2016-06-07' '2018-05-14'
 '2015-11-23' '2016-03-25' '2016-10-27' '2017-04-27' '2018-10-25'
 '2016-11-02' '2018-01-05' '2019-07-25' '2018-10-22' '2017-10-19'
 '2015-05-14' '2014-01-31' '2015-01-28' '2015-07-06' '2016-01-24'
 '2017-04-11' '2013-05-09' '2017-04-23' '2016-03-01' '2016-08-15'
 '2017-05-10' '2014-08-10' '2013-07-24' '2014-02-07' '2015-07-21'
 '2016-02-04' '2016-12-12' '2013-06-04' '2014-06-01' '2015-09-24'
 '2016-10-05' '2019-02-13' '2016-11-01' '2017-05-02' '2013-09-19'
 '2018-12-07' '2018-08-22' '2018-05-30' '2015-08-26' '2016-07-06'
 '2017

DateSignedNEWInformedConsentFoDate
[nan '2015-06-29' '2015-08-11' ... '2009-09-12' '2006-01-17' '2018-09-20']
InterferonGammaReleaseAssayIGRDate
[nan '2015-09-01' '2016-05-16' '2015-01-15' '2016-02-03' '2016-07-08'
 '2015-05-04' '2016-03-03' '2015-12-23' '2016-01-22' '2015-07-22'
 '2016-03-02' '2015-08-27' '2015-08-03' '2015-11-19' '2016-04-28'
 '2016-06-07' '2016-03-24' '2016-06-22' '2016-02-26' '2015-06-24'
 '2016-06-17' '2015-02-13' '2015-04-28' '2016-05-19' '2015-05-19'
 '2015-03-23' '2015-08-13' '2016-03-31' '2016-05-18' '2015-03-26'
 '2015-05-12' '2016-06-20' '2015-09-30' '2015-08-10' '2015-03-31'
 '2015-05-07' '2015-12-17' '2015-07-24' '2016-04-18' '2016-02-24'
 '2015-11-30' '2015-04-24' '2015-01-30' '2015-02-03' '2015-06-03'
 '2015-07-06' '2015-08-17' '2015-03-30' '2015-09-15' '2015-11-07'
 '2015-04-17' '2016-01-29' '2016-03-17' '2015-05-17' '2016-07-21'
 '2016-05-06' '2016-01-06' '2015-03-16' '2015-11-16' '2015-08-19'
 '2016-06-14' '2016-04-06' '2016-01-27' '2016-03-05' '2014-

In [122]:
for i in numerical_columns:
    if len(df[i].unique())<10:
        print(i)
        print(df[i].unique())

DateResultsReportedByLabUnknow
[nan  1.]
DegreeOfRelative_1st
[ 0.  1. nan]
DegreeOfRelative_2ndor3rd
[ 1.  0. nan]
Race_AmericanIndianorAlaskaNat
[ 0. nan  1.]
Race_Asian
[ 0.  1. nan]
Race_BlackorAfricanAmerican
[ 0. nan  1.]
Race_NativeHawaiianorOtherPaci
[ 0. nan  1.]
Race_Refused
[ 0. nan  1.]
Race_Unknownornotreported
[ 1.  0. nan]
Race_White
[ 0.  1. nan]
FirstHearAboutTrialN_ExistingTri
[ 0.  1. nan]
FirstHearAboutTrialN_FamilyFrien
[ 0.  1. nan]
FirstHearAboutTrialN_Brochure
[ 0.  1. nan]
FirstHearAboutTrialN_Healthcarep
[ 1.  0. nan]
FirstHearAboutTrialN_Tollfreepho
[ 0. nan  1.]
FirstHearAboutTrialN_NewspaperMa
[ 0. nan  1.]
FirstHearAboutTrialN_Poster
[ 0. nan  1.]
FirstHearAboutTrialN_Camp
[ 0.  1. nan]
FirstHearAboutTrialN_MeetingPres
[ 0.  1. nan]
FirstHearAboutTrialN_RadioTV
[ 0. nan  1.]
FirstHearAboutTrialN_Internet
[ 0.  1. nan]
FirstHearAboutTrialN_Other
[ 0.  1. nan]
IsHeightMeasured
[nan  1.]
IsWeightMeasured
[nan  1.]
IfSpecimensNotCollec_Other
[nan  1.]
OGTTTime

In [152]:
#these columns only have  value of 0 in it. 
element_to_remove =['EBVVCAIGM','EBVVCAIGG','EBNA','CMVIGM','HEPC','CMVIGG','CMVVL','HIV', 'HEPBC','HEPB','DR3','DR4','HLA','HLAa','HLAb']

for i in element_to_remove:
    df=df.drop(i,axis=1)


In [153]:
#columns should be coverted to categorical variables 
to_convert_cat=['DateResultsReportedByLabUnknow',
            'DegreeOfRelative_1st',
            'DegreeOfRelative_2ndor3rd',
            'Race_AmericanIndianorAlaskaNat',
            'Race_Asian',
            'Race_BlackorAfricanAmerican',
            'Race_NativeHawaiianorOtherPaci',
            'Race_Refused',
            'Race_Unknownornotreported',
            'Race_White',
            'FirstHearAboutTrialN_ExistingTri',
            'FirstHearAboutTrialN_FamilyFrien',
            'FirstHearAboutTrialN_Brochure',
            'FirstHearAboutTrialN_Healthcarep',
            'FirstHearAboutTrialN_Tollfreepho',
            'FirstHearAboutTrialN_NewspaperMa',
            'FirstHearAboutTrialN_Poster',
            'FirstHearAboutTrialN_Camp',
            'FirstHearAboutTrialN_MeetingPres',
            'FirstHearAboutTrialN_RadioTV',
            'FirstHearAboutTrialN_Internet',
            'FirstHearAboutTrialN_Other',
            'IsHeightMeasured','IsWeightMeasured',
            'IfSpecimensNotCollec_Other','Rbcnotdone',
            'HemoglobinNotDone','HematocritNotDone','MCVNotDone',
            'WBCNotDone','bloodpressureDiastolicNotDone','WeightNotDone',
            'HeightNotDone']


In [154]:
#convert some numerical variables to categorical variables 
for i in to_convert_cat:
    df[i] = df[i].astype(object)


In [155]:
categorical_columns = df.select_dtypes(include=['object', 'category']).columns
numerical_columns = df.select_dtypes(include=['int', 'float']).columns
date_columns = df.select_dtypes(include=['datetime64[ns]']).columns
#statistic the columns with <50% missing value; >=50% & <=99% missing value; >99% missing value
missing_60=[]
drop_95=[]
between_50_99=[]
for i in numerical_columns:
    count_null=df[i].isnull().sum()
    print(i)
    all1=len(df[i])
    null_per=count_null/all1
    print(null_per*100,'%')
    if null_per*100<50:
        missing_60.append(i)
    if null_per*100>=50 and null_per*100<=99:
        between_50_99.append(i)
    if null_per*100>99:
        drop_95.append(i)
        df = df.drop(i, axis=1)


MaskID
0.0 %
Rbcresults
99.2276039897477 %
HemoglobinResults
99.2276039897477 %
HematocritResults
99.2276039897477 %
MCVresults
99.2276039897477 %
PlateletCountResultsNormal
99.22837599975296 %
MCHresults
99.2276039897477 %
MCHCresults
99.23571009480283 %
WBCresults
99.2276039897477 %
NeutrophilsResults
99.60550288731741 %
NeutrophilsResultsOR
99.61515301238305 %
LymphocytesResults
99.59006268721242 %
LymphocytesResultsOR
99.60395886730691 %
MonocytesResults
99.59237871722819 %
MonocytesResultsOR
99.60434487230955 %
EosinophilsResults
99.59353673223606 %
EosinophilsResultsOR
99.60434487230955 %
BasophilsResults
99.59469474724393 %
BasophilsResultsOR
99.60704690732793 %
Age
9.89137819226137 %
clinicalcenternumber
65.76290028718772 %
ScreeningSiteNumber
30.971497390606185 %
NumOfRelativesWithT1D
18.287759009356762 %
AgeOfRelativeWithT1D1_1
16.31450143593861 %
AgeOfOnsetInRelativeWithT1D1_1
13.819751103974307 %
AgeRelativeWithT1DStartedIn1_1
16.38398233641108 %
AgeOfRelativeWithT1D2_1
84.

ParticipantDateDispensedDay4_1
99.98417379489238 %
ParticipantDateDispensedYea4_1
99.98417379489238 %
ParticipantDateDispensedDay5_1
100.0 %
ParticipantDateDispensedYea5_1
100.0 %
INST-10
16.68969829848995 %
PEP120
16.68969829848995 %
GLU60
16.68969829848995 %
PEP60
16.68969829848995 %
INST0
16.68969829848995 %
INST60
16.68969829848995 %
GLU30
16.68969829848995 %
PEP30
16.68969829848995 %
INST30
16.68969829848995 %
GLU0
16.68969829848995 %
HbA1c
16.68969829848995 %
PEP0
16.68969829848995 %
INST120
16.68969829848995 %
GLU-10
16.68969829848995 %
GLU90
16.68969829848995 %
PEP-10
16.68969829848995 %
PEP90
16.68969829848995 %
GLU120
16.68969829848995 %
INST90
16.68969829848995 %
GLU15
16.68969829848995 %
PEP15
16.68969829848995 %
PEP240
16.68969829848995 %
GLU240
16.68969829848995 %
ZNT8
16.68969829848995 %
ICA
16.68969829848995 %
PEP150
16.68969829848995 %
GLU180
16.68969829848995 %
IA-2H
16.68969829848995 %
GAD65H
16.68969829848995 %
PEP180
16.68969829848995 %
GLU150
16.68969829848995 %
P

In [156]:
len(missing_60)

72

In [157]:
len(drop_95)

178

In [158]:
len(between_50_99)

14

In [159]:
# get date columns out of the categorical columns. Use the remaining categorical columns for encoding
date_col = [col for col in categorical_columns]

def separate_columns_by_date(column_names):
    
    date_columns = []
    non_date_columns = []

    # Regular expression pattern for common date-related keywords
    date_pattern = re.compile(r'date|time', re.IGNORECASE)

    for column_name in column_names:
        if date_pattern.search(column_name):
            date_columns.append(column_name)
        else:
            non_date_columns.append(column_name)

    return date_columns, non_date_columns


date_columns, non_date_columns = separate_columns_by_date(date_col)
#these two columns are also date columns
date_columns.extend(['Visit_Dt','Specimencollectionkitshipmentd'])

element_to_remove =['Visit_Dt','Specimencollectionkitshipmentd']
for i in element_to_remove:
    if i in non_date_columns:
        non_date_columns.remove(i)


print("Columns that likely represent dates:")
print(date_columns)

print("\nColumns that do not appear to represent dates:")
print(non_date_columns)


Columns that likely represent dates:
['DateResultsReportedByLabUnknow', 'Date_of_Registration', 'RememberDateOfMostRecentHbA1c', 'DateOfRegistration', 'DiabeticKetoacidosisTimeOfDiag', 'MethylprednisoneGivenDoseTimeM', 'Spring2019ConsentDateDate', 'DateOfBloodDrawDate', 'DateReportedDate', 'ScreeningDate', 'ConsentSignedDate', 'DateConsentSignedDate', 'SamplesCollectionDate', 'LastHbA1cDate', 'WrittenInformedConsentDate', 'Type1DiabetesDiagnosisDate', 'FluVaccinationDateDate', 'DateTestsRunDate', 'StudyDrugReturnedDate', 'StudyDrugDispensedDate', 'BirthDate', 'DateOfBirthDate', 'HbA1crecordDateDate', 'DateRecordedDate1', 'DateRecordedDate2', 'DateRecordedDate3', 'ParticipantDateDispensedDate1_1', 'ParticipantDateDispensedDate2_1', 'ParticipantDateDispensedDate3_1', 'MechanisticSampleCollectionDate', 'DateSignedInformedConsentFormDate', 'DateSignedNEWInformedConsentFoDate', 'InterferonGammaReleaseAssayIGRDate', 'OGTTTestStartTime', 'MMTTTestStartTime', 'TimeOfCollection', 'TimeResultsPr

In [161]:
#fill in zero for the null values in Date/Time columns
df[date_columns]=df[date_columns].fillna(0)

In [162]:
#encode all the cateogrical variables except date and time

embed=[]
for i in non_date_columns:
    print(i)
    df[i]=df[i].astype('str')
    column_name=i
    value_counts = df[column_name].value_counts()
    #print(value_counts)
    my_dict={}
    key=df[column_name].unique()
    print('unique values:',key)
    null_count = df[column_name].isnull().sum()
    #print(f"Number of null values in column '{column_name}': {null_count}")
    le = LabelEncoder()
    df[column_name] = le.fit_transform(df[column_name])
    value=df[column_name].unique()
    # Fill the dictionary with keys and values using a loop
    for k, v in zip(key, value):
        my_dict[k] = v

    dict1={'Column':column_name,'Details':my_dict}
    value_counts = df[column_name].value_counts()
   # print(value_counts)
    null_count = df[column_name].isnull().sum()
    print(f"Number of null values after preprocessing in column '{column_name}': {null_count}")
    
    embed.append(dict1)

Visit
unique values: ['Screening' '1 year' 'Inital Visit' '12 months' '18 months' '24 months'
 '30 months' '6 months' '1 year, 6 months' '2 years' '2 years, 6 months'
 '3 years' '4 years' '3 years, 6 months' '4 years, 6 months' '5 years'
 '5 years, 6 months' '6 years' '6 years, 6 months' '7 years'
 '7 years, 6 months' '8 years' '8 years, 6 months' '11 years'
 '11 years, 6 months' '12 years, 6 months' '13 years' '1 month' '2 months'
 '3 months' '7 months' '8 months' '9 months' 'Day 1' '9 years, 6 months'
 '10 years' '9 years' '36 months' '42 months' '48 months' '54 months'
 '10 years, 6 months' '60 months' '66 months' '72 months' '78 months'
 '12 years' '13 years, 6 months' '14 years' '15 years' 'nan'
 '14 years, 6 months' '15 years, 6 months']
Number of null values after preprocessing in column 'Visit': 0
_15Aug2011ConsentForResidualSa
unique values: ['nan' 'Yes' 'No' 'Unknown']
Number of null values after preprocessing in column '_15Aug2011ConsentForResidualSa': 0
WhichProtocolVersion

Number of null values after preprocessing in column 'Race_Unknownornotreported': 0
Race_White
unique values: ['0.0' '1.0' 'nan']
Number of null values after preprocessing in column 'Race_White': 0
Relationship
unique values: ['nan' 'Mother' 'Father' 'Guardian(s)']
Number of null values after preprocessing in column 'Relationship': 0
FirstHearAboutTrialN_ExistingTri
unique values: ['0.0' '1.0' 'nan']
Number of null values after preprocessing in column 'FirstHearAboutTrialN_ExistingTri': 0
FirstHearAboutTrialN_FamilyFrien
unique values: ['0.0' '1.0' 'nan']
Number of null values after preprocessing in column 'FirstHearAboutTrialN_FamilyFrien': 0
FirstHearAboutTrialN_Brochure
unique values: ['0.0' '1.0' 'nan']
Number of null values after preprocessing in column 'FirstHearAboutTrialN_Brochure': 0
FirstHearAboutTrialN_Healthcarep
unique values: ['1.0' '0.0' 'nan']
Number of null values after preprocessing in column 'FirstHearAboutTrialN_Healthcarep': 0
FirstHearAboutTrialN_Tollfreepho
unique

unique values: ['nan'
 'I have autoantibodies which means I am at risk of getting Type 1 diabetes'
 'I have Type 1 diabetes' 'I don?t know' 'I have Type 2 diabetes']
Number of null values after preprocessing in column 'BloodTestForFindOut': 0
CommunicationWhenNegAutoantibo
unique values: ['nan' 'Letter' 'Phone Call'
 'We will only notify you if you are positive for autoantibodies'
 'I don?t know']
Number of null values after preprocessing in column 'CommunicationWhenNegAutoantibo': 0
InformedTestResultWithin
unique values: ['nan' '4-6 weeks' '1 week' '3 months' '1 year']
Number of null values after preprocessing in column 'InformedTestResultWithin': 0
PermissionToControlIfTheyTestA
unique values: ['nan' 'No' 'Unknown' 'Yes']
Number of null values after preprocessing in column 'PermissionToControlIfTheyTestA': 0
participanthaveautoantibodies
unique values: ['nan' 'Yes' 'No']
Number of null values after preprocessing in column 'participanthaveautoantibodies': 0
OnlineScreeningReferralSou

Number of null values after preprocessing in column 'MultipleSclerosisEver': 0
MultipleSclerosisWithinLastYea
unique values: ['nan' 'No']
Number of null values after preprocessing in column 'MultipleSclerosisWithinLastYea': 0
OtherAutoimmuneDiseasesEver
unique values: ['nan' 'No']
Number of null values after preprocessing in column 'OtherAutoimmuneDiseasesEver': 0
OtherAutoimmuneDiseasesWithinL
unique values: ['nan' 'No' 'Yes']
Number of null values after preprocessing in column 'OtherAutoimmuneDiseasesWithinL': 0
EverParticipatedInPreventionSt
unique values: ['nan' 'No']
Number of null values after preprocessing in column 'EverParticipatedInPreventionSt': 0
ParticipateInDPT1
unique values: ['nan' 'No']
Number of null values after preprocessing in column 'ParticipateInDPT1': 0
AllergiesEver
unique values: ['nan' 'No']
Number of null values after preprocessing in column 'AllergiesEver': 0
AllergiesWithinLastYear
unique values: ['nan' 'No']
Number of null values after preprocessing in co

unique values: ['nan' 'No' 'Yes']
Number of null values after preprocessing in column 'AllergiesWithinLast6Months': 0
CancerWithinLast6Months
unique values: ['nan' 'No' 'Yes']
Number of null values after preprocessing in column 'CancerWithinLast6Months': 0
UlcerWithinLast6Months
unique values: ['nan' 'No' 'Yes']
Number of null values after preprocessing in column 'UlcerWithinLast6Months': 0
HighBPwithinLast6Months
unique values: ['nan' 'No' 'Yes']
Number of null values after preprocessing in column 'HighBPwithinLast6Months': 0
PerniciousAnemiaWithinLast6Mo
unique values: ['nan' 'No' 'Yes']
Number of null values after preprocessing in column 'PerniciousAnemiaWithinLast6Mo': 0
MultipleSclerosisWithinLast6Mo
unique values: ['nan' 'No' 'Yes']
Number of null values after preprocessing in column 'MultipleSclerosisWithinLast6Mo': 0
hyperglycemia
unique values: ['nan' 'No' 'Yes']
Number of null values after preprocessing in column 'hyperglycemia': 0
immunomodulatory
unique values: ['nan' 'No' 

unique values: ['nan' 'Normal' 'Abnormal']
Number of null values after preprocessing in column 'LymphaticsFindings': 0
SkinFindings
unique values: ['nan' 'Normal' 'Abnormal']
Number of null values after preprocessing in column 'SkinFindings': 0
NeurologicFindings
unique values: ['nan' 'Normal' 'Abnormal']
Number of null values after preprocessing in column 'NeurologicFindings': 0
OtherFindings
unique values: ['nan' 'Not Assessed' 'Abnormal']
Number of null values after preprocessing in column 'OtherFindings': 0
ReproductiveChildbearingPotent
unique values: ['nan' 'Yes' 'No']
Number of null values after preprocessing in column 'ReproductiveChildbearingPotent': 0
UrinePregnancyTest
unique values: ['nan' 'Yes']
Number of null values after preprocessing in column 'UrinePregnancyTest': 0
UrinePregnancyTestPositive
unique values: ['nan' 'No']
Number of null values after preprocessing in column 'UrinePregnancyTestPositive': 0
PlateletCountNotDone
unique values: ['nan' '  .' '1.0' '1']
Number 

unique values: ['nan' 'Yes' 'No']
Number of null values after preprocessing in column 'Wasaurinepregnancytestcomplete': 0
Wasthetestresultpositive
unique values: ['nan' 'No']
Number of null values after preprocessing in column 'Wasthetestresultpositive': 0
bloodpressureDiastolicNotDone
unique values: ['nan' '1.0']
Number of null values after preprocessing in column 'bloodpressureDiastolicNotDone': 0
WeightNotDone
unique values: ['nan' '1.0']
Number of null values after preprocessing in column 'WeightNotDone': 0
HeightNotDone
unique values: ['nan' '1.0']
Number of null values after preprocessing in column 'HeightNotDone': 0
Wasaphysicalexamperformedatthi
unique values: ['nan' 'Yes' 'No']
Number of null values after preprocessing in column 'Wasaphysicalexamperformedatthi': 0
HEENT
unique values: ['nan' 'Normal' 'Abnormal' 'Not Assessed']
Number of null values after preprocessing in column 'HEENT': 0
NeckThyroid
unique values: ['nan' 'Normal' 'Not Assessed' 'Abnormal']
Number of null valu

In [163]:
embed

[{'Column': 'Visit',
  'Details': {'Screening': 51,
   '1 year': 1,
   'Inital Visit': 50,
   '12 months': 7,
   '18 months': 16,
   '24 months': 20,
   '30 months': 24,
   '6 months': 33,
   '1 year, 6 months': 2,
   '2 years': 18,
   '2 years, 6 months': 19,
   '3 years': 22,
   '4 years': 26,
   '3 years, 6 months': 23,
   '4 years, 6 months': 27,
   '5 years': 30,
   '5 years, 6 months': 31,
   '6 years': 34,
   '6 years, 6 months': 35,
   '7 years': 39,
   '7 years, 6 months': 40,
   '8 years': 44,
   '8 years, 6 months': 45,
   '11 years': 5,
   '11 years, 6 months': 6,
   '12 years, 6 months': 9,
   '13 years': 10,
   '1 month': 0,
   '2 months': 17,
   '3 months': 21,
   '7 months': 38,
   '8 months': 43,
   '9 months': 46,
   'Day 1': 49,
   '9 years, 6 months': 48,
   '10 years': 3,
   '9 years': 47,
   '36 months': 25,
   '42 months': 28,
   '48 months': 29,
   '54 months': 32,
   '10 years, 6 months': 4,
   '60 months': 36,
   '66 months': 37,
   '72 months': 41,
   '78 mon

In [164]:
#save it to create data dictionary
with open('encoding.pickle', 'wb') as file:
    pickle.dump(embed, file)

In [166]:
len(missing_60)

72

In [167]:
len(drop_95)

178

In [168]:
len(between_50_99)

14

In [170]:
between_50_99

['clinicalcenternumber',
 'AgeOfRelativeWithT1D2_1',
 'AgeOfOnsetInRelativeWithT1D2_1',
 'AgeRelativeWithT1DStartedIn2_1',
 'AgeOfRelativeWithT1D3_1',
 'AgeOfOnsetInRelativeWithT1D3_1',
 'AgeRelativeWithT1DStartedIn3_1',
 'AgeOfRelativeWithT1D4_1',
 'AgeOfOnsetInRelativeWithT1D4_1',
 'AgeRelativeWithT1DStartedIn4_1',
 'SeatedArmBPSystolic',
 'SeatedArmBPDiastolic',
 'SeatedHeartRate',
 'SeatedRespiratoryRate']

In [172]:
# Loop through the specified columns and add a missing indicator column for each
# 1 means misssing , 0 means not missing
for col in between_50_99:
    df[f'{col}_missing_indicator'] = df[col].isnull().astype(int)

/tmp/ipykernel_33758/1734350622.py:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[f'{col}_missing_indicator'] = df[col].isnull().astype(int)
/tmp/ipykernel_33758/1734350622.py:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[f'{col}_missing_indicator'] = df[col].isnull().astype(int)
/tmp/ipykernel_33758/1734350622.py:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis

In [184]:
len(missing_60)

72

In [ ]:
# Initialize the KNN imputer
imputer = KNNImputer(n_neighbors=5)  # You can adjust 'n_neighbors' as needed
# Impute missing values using the KNN imputer for each numeric column with less than 50% missing value
df[missing_60] = imputer.fit_transform(df[missing_60])

In [15]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 259064 entries, 0 to 259063
Columns: 473 entries, MaskID to SeatedRespiratoryRate_missing_indicator
dtypes: float64(86), int64(349), object(38)
memory usage: 934.9+ MB


In [ ]:
df.to_csv('AI_Ready.csv',index=False)